# Data Formats

In this tutorial, we will cover several manipulations of data formats:
- TIFF
- NetCDF4
- HDF5
- Zarr
- Pickle
- JSON
- CSV vs Parquet (practice again)

In [1]:
import requests, zipfile , os, io
import folium
import geopandas as gpd
import matplotlib.pyplot as plt
# import netCDF4 as nc
import numpy as np
import pandas as pd
import pycrs
import rasterio
import h5py
import rasterio
import netCDF4 as nc
import wget


from folium.plugins import MarkerCluster
from rasterio.mask import mask
from rasterio.plot import show

ModuleNotFoundError: No module named 'wget'

## Raster data: rasterio

Raster data is any pixelated (or gridded) data where each pixel is associated with a specific geographical location. The value of a pixel can be continuous (e.g. elevation) or categorical (e.g. land use).



https://rasterio.readthedocs.io/en/latest/

https://www.earthdatascience.org/courses/use-data-open-source-python/intro-raster-data-python/

https://geohackweek.github.io/raster/

We will download topography files from there (data saved onto a Dropbox folder)

https://www.naturalearthdata.com/downloads/50m-raster-data/50m-cross-blend-hypso/



In [ ]:
# Dowload the data using wget
wget.download("https://www.dropbox.com/s/r75ecms0bvyqaca/HYP_50M_SR.zip?dl=0")

Unzip the file

In [5]:

# url='https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/50m/raster/HYP_50M_SR.zip'
# s = requests.get(url) 
os.makedirs("./HYP_50M_SR/",exist_ok=True)
# wget.download(url,out="HYP_50M_SR") # this does not work on the hub
z = zipfile.ZipFile("HYP_50M_SR.zip")
z.extractall('./HYP_50M_SR/')

In [6]:
elevation = rasterio.open('HYP_50M_SR/HYP_50M_SR.tif')

In [9]:
print(elevation)

<open DatasetReader name='HYP_50M_SR/HYP_50M_SR.tif' mode='r'>


Let us have a look at the dimensions of the data:

In [7]:
elevation.height

5400

In [8]:
elevation.width

10800

In [ ]:
elevation.indexes

Can you guess on how to call the data types of the file entry?

In [ ]:
# type below

and at the boundaries of the dataset:

In [ ]:
elevation.bounds

In [ ]:
print(elevation.transform * (0, 0)) # North West corner
print(elevation.transform * (elevation.width, elevation.height)) # South East corner

Here is the projection used for the data:

In [ ]:
elevation.crs

How to interpret the data: There are three layers for the three colors red, green, and blue:

In [ ]:
print(elevation.colorinterp[0])
print(elevation.colorinterp[1])
print(elevation.colorinterp[2])

In [ ]:
print(np.min(elevation.read(1)), np.max(elevation.read(1)))
print(np.min(elevation.read(2)), np.max(elevation.read(2)))
print(np.min(elevation.read(3)), np.max(elevation.read(3)))

Let us now plot the data:

In [ ]:
image = elevation.read()

In [ ]:
show(image)

##  geojson

Shapefiles are not the only kind of files that we can read with geopandas. Let us look at an example of reading data from a geojson file (which is a special case of json file with geographical coordinates).

You can find more about json in:

https://www.earthdatascience.org/courses/use-data-open-source-python/intro-to-apis/apis-in-python/

In [ ]:
url = 'https://www.nps.gov/lib/npmap.js/4.0.0/examples/data/national-parks.geojson'

In [ ]:
parks = gpd.read_file(url)

In [ ]:
parks.head()

Let us plot the data.

folium is a nice Python package for visualization. You can learn more in the following notebook:

https://github.com/geohackweek/tutorial_contents/blob/master/visualization/notebooks/foliumTutorial.ipynb

In [ ]:
m = folium.Map(location=[40, -100], zoom_start=4)
folium.GeoJson(parks).add_to(m)
marker_cluster = MarkerCluster().add_to(m)
m

We are going to focus on the parks in Washington State:

In [ ]:
parks_WA = parks.iloc[[94, 127, 187, 228, 286, 294, 295, 297, 299, 300, 302]].reset_index()

We create a list of locations to add popups to the map.

In [ ]:
locations = []
for index in range(0, len(parks_WA)):
    location = [parks_WA['geometry'][index].y, parks_WA['geometry'][index].x]
    locations.append(location)

In [ ]:
m = folium.Map(location=[47, -121], zoom_start=7)
marker_cluster = MarkerCluster().add_to(m)
for point in range(0, len(locations)):
    folium.Marker(location = locations[point], popup=parks_WA['Name'].iloc[point]).add_to(marker_cluster)
m

## Hierarchical formats: NETCDF4 & HDF5

Hierarchical data formats are designed to store large amount of data into a single file. They mimic a file system (e.g., tree-like data structure with nested directories) into a single file.  There are two domiment hierarchical data formats (HDF5 and NETCDF4), and one emerging for cloud (Zarr). Hierarchical formats in general can store many data types (numeric vs string).

## HDF5
The Hierarchical Data Format version 5 (HDF5), is an open source file format that supports large, complex, heterogeneous data. HDF5 uses a "file directory" like structure that allows you to organize data within the file in many different structured ways, as you might do with files on your computer. The HDF5 format also allows for embedding of metadata making it _self-describing_.
HDF5 files are self describing - this means that all elements (the file itself, groups and datasets) can have associated metadata that describes the information contained within the element.


HDF Structure Example:
- Datasets, which are typed multidimensional arrays
- Groups, which are container structures that can hold datasets and other groups


<img src="hdf5_structure4.jpeg" alt="An illustration of a H5 data set " />
Figure: HDF5 data example. Found in [neonscience](https://www.neonscience.org/resources/learning-hub/tutorials/about-hdf5)

## Netcdf

The network Common Data Form, or **netCDF**, was created in the early 1990s, and set out to solve some of the challenges in working with N-dimensional arrays. Netcdf is a collection of self-describing, machine-independent binary data formats and software tools that facilitate the creation, access and sharing of scientific data stored in N-dimensional arrays, along with metadata describing the contents of each array. Netcdf was built by the climate science community at a time when regional climate models were beginning to produce larger and larger output files. NetCDF version 4 is now a subset of HDF5.


### Handling large arrays
The NetCDF & H5 format have no limit on file sizes. However, any analysis tools that read data from a NetCDF array into memory for some computational operation will be limited by that particular machine's available memory. 

### But slow at I/O
When reading a hierarchical file, the whole tree of the data structure is scanned from the root node down. Since it has to be done each time a user makes an inquiry, reading H5 and Netcdf is **slow**. There are faster



The data files for this part can be downloaded here:

https://www.sciencebase.gov/catalog/item/5cfeb4cce4b0156ea5645056

In [ ]:
file1 = 'NCM_GeologicFrameworkGrids.nc'
file2 = 'NCM_SpatialGrid.nc'
geology = nc.Dataset(file1)
grid = nc.Dataset(file2)

In [ ]:
geology

In [ ]:
geology['Surface Elevation']

In [ ]:
np.shape(geology['Surface Elevation'])

In [ ]:
geology['Surface Elevation'][3246, 1234]

In [ ]:
x = grid['x'][0:4901, 0:3201]
y = grid['y'][0:4901, 0:3201]
elevation = geology['Surface Elevation'][0:4901, 0:3201]

In [ ]:
plt.contourf(x, y, elevation)

## Zarr


Zarr is a cloud-optimized data format that handles heterogeneous data sets.

In the following exercise, we will use the Xarray open data sets air_temperature and save it into a Zarr file.

Let's work in groups to :
- Download the xarray data
- Save it in to file, report on time and size of the data set.
- Read it again and check again write time and read times